# Hands-On Demo: Building a Web-Browsing Agent

#### Install Playwright

playwright, nest_asyncio

##### For Mac and Windows
`playwright install`

##### For Linux
`playwright install —with-reps chromium`

In [ ]:
# Handle Async in Notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)

tools = toolkit.get_tools()

for tool in tools:
    print(tool.name," :: ", tool.description)


## AI Web Assistant – LangGraph + Playwright + MemorySaver + Gradio

In [ ]:
import nest_asyncio
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from bs4 import BeautifulSoup
import textwrap
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


nest_asyncio.apply()
async_browser =  create_async_playwright_browser(headless=True)  # headful mode
toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)

tools = toolkit.get_tools()

# Define State 
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize Graph
graph_builder = StateGraph(State)

# Playwright Browser Toolkit
# tool_dict = {tool.name:tool for tool in tools}
# navigate_tool = tool_dict.get("navigate_browser")
# extract_text_tool = tool_dict.get("extract_text")

# await navigate_tool.arun({"url": "https://www.bbc.com"})
# text = await extract_text_tool.arun({})
# print(textwrap.fill(text, width=100))

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")
llm = llm.bind_tools(tools)

# Define the Node
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# Add Nodes and Edges
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Compile the Graph
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
import gradio as gr

config = {"configurable": {"thread_id": "1"}}

async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()